In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt
import optuna
import pickle
import joblib
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore')

/Users/cash/Desktop/F5/Python/DataScience/data-scientist-grupo4/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cargar los datos
df = pd.read_csv('stroke_dataset.csv')

In [3]:
# Define las columnas
cat_cols = ['gender', 'ever_married', 'Residence_type', 'work_type', 'smoking_status']  # Todas las variables categóricas
num_cols = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']  # Variables numéricas

In [4]:

# Aplicar LabelEncoder a cada columna categórica
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Preprocesamiento simplificado usando LabelEncoder
preprocessor = ColumnTransformer(
    transformers=[
        # Escalar las variables numéricas
        ('num', StandardScaler(), num_cols)
    ],
    remainder='passthrough'  # Mantener las columnas categóricas ya codificadas
)

In [5]:
# Separar características y objetivo
X = df[cat_cols + num_cols]
y = df['stroke']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Función para crear el pipeline con SMOTE
def create_pipeline(classifier):
    return ImbPipeline([
        ('preprocessor', preprocessor), # preprocesamiento
        ('smote', SMOTE(random_state=42)), # Synthetic Minority Over-sampling Technique
        ('classifier', classifier) # clasificador
    ])

In [7]:
# Función para evaluar el modelo
def evaluate_model(y_true, y_pred, y_pred_proba):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc': roc_auc_score(y_true, y_pred_proba)
    }

In [8]:
# Diccionario de modelos
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'LightGBM': LGBMClassifier(),
    'XGBoost': XGBClassifier(),
    'CatBoost': CatBoostClassifier(verbose=0)
}

In [9]:
# Entrenar y evaluar modelos
results = {}
best_model = None
best_auc = 0

for name, model in models.items():
    pipeline = create_pipeline(model)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    metrics = evaluate_model(y_test, y_pred, y_pred_proba)
    results[name] = metrics
    
    print(f"\n{name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    if metrics['auc'] > best_auc:
        best_auc = metrics['auc']
        best_model = (name, pipeline)

print(f"\nBest model: {best_model[0]} with AUC = {best_auc:.4f}")


Logistic Regression:
accuracy: 0.7362
precision: 0.1433
recall: 0.7778
f1: 0.2421
auc: 0.8445

Random Forest:
accuracy: 0.9097
precision: 0.0263
recall: 0.0185
f1: 0.0217
auc: 0.7818
[LightGBM] [Info] Number of positive: 3790, number of negative: 3790
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2051
[LightGBM] [Info] Number of data points in the train set: 7580, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

LightGBM:
accuracy: 0.9137
precision: 0.1000
recall: 0.0741
f1: 0.0851
auc: 0.7732

XGBoost:
accuracy: 0.9208
precision: 0.0968
recall: 0.0556
f1: 0.0706
auc: 0.7741

CatBoost:
accuracy: 0.9188
precision: 0.1143
recall: 0.0741
f1: 0.0899
auc: 0.7418

Best model: Logistic Regression with AUC = 0.8445


In [10]:
# Optimización de hiperparámetros con Optuna
def objective(trial):
    if best_model[0] == 'Logistic Regression':
        solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear'])
        
        if solver == 'liblinear':
            penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
        else:
            penalty = 'l2'  # lbfgs solo soporta 'l2' o None
        
        classifier = LogisticRegression(
            C=trial.suggest_loguniform('C', 1e-5, 1e5),
            solver=solver,
            penalty=penalty,
            random_state=42
        )
    elif best_model[0] == 'Random Forest':
        classifier = RandomForestClassifier(
            n_estimators=trial.suggest_int('n_estimators', 10, 1000),
            max_depth=trial.suggest_int('max_depth', 2, 32),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 20)
        )
    elif best_model[0] == 'LightGBM':
        classifier = LGBMClassifier(
            num_leaves=trial.suggest_int('num_leaves', 2, 256),
            max_depth=trial.suggest_int('max_depth', 3, 16),
            learning_rate=trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
            n_estimators=trial.suggest_int('n_estimators', 100, 1000)
        )
    elif best_model[0] == 'XGBoost':
        classifier = XGBClassifier(
            max_depth=trial.suggest_int('max_depth', 1, 9),
            learning_rate=trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
            n_estimators=trial.suggest_int('n_estimators', 100, 1000),
            min_child_weight=trial.suggest_int('min_child_weight', 1, 300)
        )
    else:  # CatBoost
        classifier = CatBoostClassifier(
            iterations=trial.suggest_int('iterations', 100, 1000),
            depth=trial.suggest_int('depth', 4, 10),
            learning_rate=trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
            l2_leaf_reg=trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
            verbose=0
        )
    
    pipeline = create_pipeline(classifier)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []
    
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        pipeline.fit(X_train_fold, y_train_fold)
        y_val_pred_proba = pipeline.predict_proba(X_val_fold)[:, 1]
        
        auc = roc_auc_score(y_val_fold, y_val_pred_proba)
        auc_scores.append(auc)
    
    return np.mean(auc_scores)

In [11]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Entrenar el mejor modelo con los mejores hiperparámetros
best_params = study.best_params
best_classifier = models[best_model[0]]
best_classifier.set_params(**best_params)
best_pipeline = create_pipeline(best_classifier)
best_pipeline.fit(X_train, y_train)

[I 2024-10-24 20:26:09,573] A new study created in memory with name: no-name-bec6e09d-84fe-4a07-9469-8a96d6ac4562
[I 2024-10-24 20:26:09,779] Trial 0 finished with value: 0.8291966215518501 and parameters: {'solver': 'lbfgs', 'C': 69.627591301926}. Best is trial 0 with value: 0.8291966215518501.
[I 2024-10-24 20:26:10,027] Trial 1 finished with value: 0.8291966215518501 and parameters: {'solver': 'lbfgs', 'C': 69.67804855564168}. Best is trial 0 with value: 0.8291966215518501.
[I 2024-10-24 20:26:10,190] Trial 2 finished with value: 0.8292039211078233 and parameters: {'solver': 'lbfgs', 'C': 0.24488619524991453}. Best is trial 2 with value: 0.8292039211078233.
[I 2024-10-24 20:26:10,319] Trial 3 finished with value: 0.8163677409476605 and parameters: {'solver': 'lbfgs', 'C': 5.680748690559047e-05}. Best is trial 2 with value: 0.8292039211078233.
[I 2024-10-24 20:26:10,580] Trial 4 finished with value: 0.8291013712482954 and parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 328.

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['age', 'hypertension',
                                                   'heart_disease',
                                                   'avg_glucose_level',
                                                   'bmi'])])),
                ('smote', SMOTE(random_state=42)),
                ('classifier',
                 LogisticRegression(C=0.01324907225420572, penalty='l1',
                                    solver='liblinear'))])

In [12]:
# Evaluar el modelo optimizado
y_pred_optimized = best_pipeline.predict(X_test)
y_pred_proba_optimized = best_pipeline.predict_proba(X_test)[:, 1]
metrics_optimized = evaluate_model(y_test, y_pred_optimized, y_pred_proba_optimized)

print(f"\nBest model after optimization: {best_model[0]}")
print(f"Best parameters: {best_params}")
print("\nOptimized model metrics:")
for metric, value in metrics_optimized.items():
    print(f"{metric}: {value:.4f}")


Best model after optimization: Logistic Regression
Best parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.01324907225420572}

Optimized model metrics:
accuracy: 0.7222
precision: 0.1438
recall: 0.8333
f1: 0.2452
auc: 0.8464


In [13]:
# Calcular e imprimir el overfitting
y_train_pred_proba = best_pipeline.predict_proba(X_train)[:, 1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)
overfitting = train_auc - metrics_optimized['auc']
print(f"\nTrain AUC: {train_auc:.4f}")
print(f"Overfitting: {overfitting:.4f}")


Train AUC: 0.8410
Overfitting: -0.0054


In [14]:
# Guardar el pipeline completo en un archivo pickle
joblib.dump(best_pipeline, 'full_stroke_prediction_pipeline.pkl')
print(f"\nFull pipeline saved as: 'full_stroke_prediction_pipeline.pkl'")


Full pipeline saved as: 'full_stroke_prediction_pipeline.pkl'
